# Signals

Most evaluations of carbon-aware approaches rely on some kind of time-stepping simulation over historical traces of carbon intensity or power production/consumption. Vessim's `Signal` simplifies the implementation of such evaluations that do not require the simulation of entire microgrids. Additionally, it's used to model grid level signals or the behavior of `Actors` in co-simulations.

Vessim currently ships a `HistoricalSignal` implementation which replays datasets that are usually loaded from CSV files. We are working on live data signals to common APIs such as [WattTime](https://watttime.org/>), [Electricity Maps](https://www.electricitymaps.com), or [Solcast](https://solcast.com>).


In [1]:
import pandas as pd
import vessim as vs

## Basic usage

The `HistoricalSignal` can load datasets from pandas DataFrames, where the index describes the timestamps and each column a certain value.

In [2]:
actual = pd.DataFrame([
   ["2020-01-01T00:00:00", 100, 324],
   ["2020-01-01T00:30:00", 110, 357],
   ["2020-01-01T01:00:00", 105, 398],
], columns=["timestamp", "solar", "wind"])
actual = actual.set_index(["timestamp"])
signal = vs.HistoricalSignal(actual)

In [3]:
signal.now(at="2020-01-01T00:00:00", column="solar")

100.0

Additionally, the `HistoricalSignal` allows specifying forecasts with a 2-level multiindex, where the first column describes the time the forecast was queried and the second column the target time.

In [4]:
forecast = pd.DataFrame([
   ["2020-01-01T00:00:00", "2020-01-01T00:30:00", 115, 370],
   ["2020-01-01T00:00:00", "2020-01-01T01:00:00", 108, 372],
   ["2020-01-01T00:30:00", "2020-01-01T01:00:00", 109, 372],
   ["2020-01-01T00:30:00", "2020-01-01T01:30:00", 102, 378],
   ["2020-01-01T01:00:00", "2020-01-01T01:30:00", 101, 382],
   ["2020-01-01T01:00:00", "2020-01-01T02:00:00",  88, 398],
], columns=["req_time", "forecast_time", "solar", "wind"])
forecast = forecast.set_index(["req_time", "forecast_time"])
signal = vs.HistoricalSignal(actual, forecast)

In [5]:
signal.forecast("2020-01-01T00:00:00", end_time="2020-01-01T01:00:00", column="solar")

{numpy.datetime64('2020-01-01T00:30:00.000000000'): 115.0,
 numpy.datetime64('2020-01-01T01:00:00.000000000'): 108.0}

For more advanced features such as configuring the interpolation between values that are not present in the original dataset, please refer to the API reference of the documentation.

## Datasets

Vessim comes with several datasets that can be directly loaded into a `HistoricalSignal` with the possibility of shifting and scaling. For example, `solcast2022_global` and `solcast2022_germany` are solar irradiance datasets provided by [Solcast](https://solcast.com>) as described in [this paper](https://arxiv.org/pdf/2305.15092.pdf).

In [6]:
dataset_signal = vs.HistoricalSignal.load("solcast2022_global", params={"scale": 2.0, "start_time":"2020-06-12T00:00:00"})
dataset_signal.now(at="2020-06-27T13:46:53", column="Berlin")

Data is being shifted by -727 days +23:55:00


1.289502

Further documentation of available datasets can be found on our website.